In [ ]:
#Copy data to local scratch
import os
datasetName = "geler"
datasetFrom = "/home/xca64/remote/GitHub/colorP/dataSet/" + datasetName
datasetTo = "/local-scratch/xca64/"
cpCommand = "cp -rf " + datasetFrom + " " + datasetTo 
rmCommand = "rm -rf " +  datasetTo + datasetName 
print cpCommand 
os.system(cpCommand)

In [1]:
import os
os.chdir('../../')
import sys
sys.path.insert(0, './python')
sys.path.append('/local-scratch/xca64/tmp/caffe-master/python/myFunc')
import caffe
import numpy as np
from pylab import *
%matplotlib inline

In [2]:
niter = 200
# losses will also be stored in the log
train_loss = np.zeros(niter)
scratch_train_loss = np.zeros(niter)

caffe.set_device(0)
caffe.set_mode_gpu()

In [3]:
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver.prototxt')
solver.net.copy_from('models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

In [4]:
import tempfile

In [7]:
def run_solver(solver, niter, disp_interval):
    blobs = ('loss', 'acc')
    loss, acc = (np.zeros(niter), np.zeros(niter))
    for it in range(niter):
        solver.step(1)  # run a single SGD step in Caffe
        loss[it] = (solver.net.blobs['loss'].data.copy())
        acc[it] = 0#(solver.net.blobs['loss_ang'].data.copy())
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = 'loss: %.3f'%loss[it]
            print '%3d) %s Angular Erro %.3f' % (it, loss_disp, acc[it])     
            print(solver.net.blobs['fc9'].data[1], solver.net.blobs['illu'].data[1])
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    name = 'firstTry'
    weights = {}
    filename = 'weights.%s.caffemodel' % name
    
    weights[name] = os.path.join(weight_dir, filename)
    solver.net.save(weights[name])
    
    return loss, acc, weights

In [8]:
loss_1, acc_1, weights_1 = run_solver(solver, 100,10)

  0) loss: 41.984 Angular Erro 0.000
(array([ 0.05942917,  0.01559612,  0.03532755], dtype=float32), array([ 0.1022988,  0.092635 ,  0.0841419], dtype=float32))
 10) loss: 3.274 Angular Erro 0.000
(array([ 0.50264037,  0.46024597,  0.40114674], dtype=float32), array([ 0.0860871,  0.0779288,  0.0730716], dtype=float32))
 20) loss: 2.546 Angular Erro 0.000
(array([ 1.60841   ,  1.38864648,  1.17414236], dtype=float32), array([ 0.0426969,  0.0368868,  0.0349014], dtype=float32))
 30) loss: 3.275 Angular Erro 0.000
(array([ 1.59596837,  1.42029691,  1.22112751], dtype=float32), array([ 0.0809324,  0.0590289,  0.0450431], dtype=float32))
 40) loss: 4.270 Angular Erro 0.000
(array([ 1.99348581,  1.70642638,  1.5401814 ], dtype=float32), array([ 0.15867279,  0.1136707 ,  0.0871683 ], dtype=float32))
 50) loss: 4.124 Angular Erro 0.000
(array([ 1.96735454,  1.74125051,  1.61262214], dtype=float32), array([ 0.1106784,  0.0912051,  0.0781661], dtype=float32))
 60) loss: 2.913 Angular Erro 0.000


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [ ]:
#print loss_1
#plot the trainning los
plot(np.vstack([loss_1]).clip(0, 10).T)
solver.net.save('models/color_constancy/result/gehler_482_3000_iters.caffemodel')

In [ ]:
def deprocess_net_image(image, gamma):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255
    
    image = image.astype(np.float32)
    image = image/255
    image = image**gamma #Gamma correction
    image = image*255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image


In [ ]:
def my_deprocess_net_image(image, gamma):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    #image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255
    
    image = image.astype(np.float32)
    image = image/255
    image = image**gamma #Gamma correction
    image = image*255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image

In [ ]:
solver.step(1)  # SGD by Caffe

In [ ]:
for i in range(0,5):
    img = my_deprocess_net_image(solver.net.blobs['data'].data[i], 0.2)
    print solver.net.blobs['illu'].data[i]
    plt.figure(i)
    plt.imshow(img)


In [ ]:
for i in range(0,5):
    img = deprocess_net_image(solver.net.blobs['data'].data[i], 1)
    print solver.net.blobs['illu'].data[i]
    plt.figure(i)
    plt.imshow(img)

In [ ]:
print np.min(solver.net.blobs['data'].data[1])
print solver.net.blobs['data'].num

In [ ]:
os.system(rmCommand)

In [ ]:
# We run the solver for niter times, and record the training loss.
for it in range(niter):
    solver.step(1)  # SGD by Caffe
    # store the train loss
    train_loss[it] = solver.net.blobs['loss'].data
    if it % 10 == 0:
        print 'iter %d, finetune_loss=%f, scratch_loss=' % (it, train_loss[it])
print 'done'